---
# 3.2 Bisection method
---

Suppose $f \in C[a,b]$ and that $f(a)$ and $f(b)$ have opposite signs; i.e.,

$$
f(a) \cdot f(b) < 0.
$$

Recall the following important theorem from calculus.

> ### Intermediate Value Theorem
>
> If $f \in C[a,b]$ and $f(a) \leq s \leq f(b)$, then there exists a real number $c \in [a,b]$ such that $f(c) = s$.

Since $f$ changes sign over $[a,b]$, the Intermediate Value Theorem implies that there is some $x^* \in [a,b]$ such that $f(x^*) = 0$.

The **bisection method** searches for a root of $f$ in $[a,b]$ as follows.

1. Let $p = (a+b)/2$ be the **midpoint** of $[a,b]$.
2. If $f(a) \cdot f(p) < 0$, then there is a root in $[a,p]$.
3. If $f(a) \cdot f(p) = 0$, then $p$ is a root.
4. If $f(a) \cdot f(p) > 0$, then there is a root in $[p,b]$.

Each time we apply the above, we get a subinterval that contains a root that is **half the size** of the interval $[a,b]$.

In [ ]:
using Printf

# Code the bisection method
function bisect(f, a, b, tol=1e-8)
    fa, fb = f(a), f(b)
    
    if fa == 0.0
        return a
    elseif fb == 0.0
        return b
    end
        
    if fa*fb > 0
        error("The sign of f(a) and f(b) must be different.")
    end
    
    k = 0
    p = (a + b)/2
    oldp = Inf
    @printf "%4s %12s %12s %12s\n" "k" "a" "p" "b"
    @printf "%4d %12.8f %12.8f %12.8f\n" k a p b
    while abs(p - oldp) > tol*(1 + abs(p))
        k += 1
        fp = f(p)
        if abs(fp) < tol
            return p
        end

        if fa*fp < 0
            b = p
        else
            a = p
        end
        oldp = p
        p = (a + b)/2
        @printf "%4d %12.8f %12.8f %12.8f\n" k a p b
    end
    
    return p
    
end

In [ ]:
f(x) = x - 2

In [ ]:
bisect(f, 0., 1.)

In [ ]:
bisect(f, 1., 2.)

In [ ]:
bisect(f, 1., 3.)

In [ ]:
xs = 2.0

rt = bisect(f, 1., 100.)

In [ ]:
abs(rt - xs)

---

### Example

We will now use the above `bisect` function to find the root of

$$
f(x) = 2 \cosh(x/4) - x
$$

that lies in the interval $[5,10]$.

In [ ]:
rt = bisect(x -> 2cosh(x/4) - x, 5., 10.)

In [ ]:
2cosh(rt/4) - rt

---

### Example

Let's try to find the value of $y_0$ such that the solution $y(t)$ of the ordinary differential equation 

$$
\frac{dy}{dt} = t - 5 + \frac{y^2}{100}
$$

with initial value $y(0) = y_0$ satisfies $y(10) = 20$.

In [ ]:
using OrdinaryDiffEq  # Package for solving ordinary differential equations
using Plots

In [ ]:
F(y, p, t) = t - 5 + y^2/100

In [ ]:
tspan = (0.0, 10.0)

y0vals = [5, 10, 15]
plt = plot(legend=:topleft)
for tt = enumerate(y0vals)
    n, y0 = tt
    prob = ODEProblem(F, y0, tspan)
    sol = solve(prob, Tsit5())
    yval = round(sol(10), digits=2)
    plot!(sol, label="y(10) = $yval", color=n)
    scatter!([0.0, 10.0], [y0, sol(10.0)], color=n, label=:none)
end
scatter!([10.0], [20.0], color=4, label=:none)
xlims!(-2, 12)
plt

In [ ]:
function foo(y0)
    tspan = (0.0, 10.0)
    prob = ODEProblem(F, y0, tspan)
    sol = solve(prob, Tsit5())
    return sol(10.0)
end

In [ ]:
@time foo(5.)

In [ ]:
@time foo(5.)

In [ ]:
foo(10.)

In [ ]:
foo(15.)

In [ ]:
@time y0 = bisect(y0 -> foo(y0) - 20, 10., 15.)

In [ ]:
foo(y0)

In [ ]:
tspan = (0.0, 10.0)
prob = ODEProblem(F, y0, tspan)
sol = solve(prob, Tsit5())
yval = round(sol(10), digits=2)
plot!(sol, label="y(10) = $yval", color=4)
scatter!([0.0, 10.0], [y0, sol(10.0)], color=4, label=:none)
xlims!(-2, 12)

---

## Analyzing the bisection method

Initially, we know a root $x^*$ is somewhere in the interval $[a,b]$. If we let $x_k$ be the midpoint of the $k$th subinterval, then

$$\left|x^* - x_0\right| \leq \frac{b-a}{2}.$$

In the next iteration, 

$$\left|x^* - x_1\right| \leq \frac{b-a}{4},$$

and in the following iteration,

$$\left|x^* - x_2\right| \leq \frac{b-a}{8},$$

and so on, each time reducing our error by a factor of $2$.

In general,

$$\left|x^* - x_k\right| \leq \frac{b-a}{2} \cdot 2^{-k}, 
\qquad \text{for $k = 0,1,2,\ldots$}.$$

Suppose we want to compute $x_k$ such that 

$$\left|x^* - x_k\right| \leq \mathtt{atol}.$$

Then we just need to find the smallest positive integer $k$ such that

$$\frac{b-a}{2} \cdot 2^{-k} \leq \mathtt{atol}.$$

That is,

$$\frac{b-a}{2\mathtt{atol}} \leq 2^k,$$

which gives us

$$\log_2\left(\frac{b-a}{2\mathtt{atol}}\right) \leq k,$$

so we just need the first integer $k$ that is larger than $\log_2\left(\frac{b-a}{2\mathtt{atol}}\right)$. Therefore, 

$$k = \left\lceil \log_2\left(\frac{b-a}{2\mathtt{atol}}\right) \right\rceil.$$

In [ ]:
atol = 1e-8
a, b = 10., 15.

k = ceil(log2((b - a)/2atol))

---

## Pros and cons of the bisection method

Pros:

1. **Simple:** The bisection method only requires function values, is easy to understand and implement, and it is easy to analyze.

2. **Robust:** The bisection method is guaranteed to work, provided that $f$ is continuous and changes sign on the interval $[a,b]$.

Cons:

1. **Slow to converge:** The bisection method often requires many function evaluations.

2. **Does not generalize:** The bisection method only applies to solving equations involving one variable; it does not generalize to solving equations involving multiple variables.



---